<a href="https://colab.research.google.com/github/SowjanyaKiran/Encoder_Decoder_Architecture/blob/main/Encoder_Decoder_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Please connect to v5e-1 TPU runtime

In [ ]:
#Installing required libraries
!pip install tensorflow tensorflow_datasets tfds

In [ ]:
#Importing important libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , LSTM , Embedding , Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Loading dataset from Tnsorflow dataset
examples , metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)
train_examples , val_examples = examples['train'] , examples['validation']

In [ ]:
#We'll only use a small subset for demonstration
MAX_SAMPLES = 1000

In [ ]:
#Function for preprocessing the data
def preprocess(data , num_samples = MAX_SAMPLES):
  src_texts = []
  tgt_texts = []

  for src , tgt in data.take(num_samples):
    src_texts.append(src.numpy().decode('utf-8'))
    tgt_texts.append(tgt.numpy().decode('utf-8'))
    tgt = '<start> ' + tgt + ' <end>'

  return src_texts , tgt_texts

In [ ]:
#Driver code
input_texts , target_texts = preprocess(train_examples)

In [ ]:
#Tokenization
def tokenize(sentences):
  tokenizer = Tokenizer(filters = '')
  tokenizer.fit_on_texts(sentences)
  tensor = tokenizer.texts_to_sequences(sentences)
  tensor = pad_sequences(tensor , padding = 'post' , maxlen = 20 , truncating = 'post')
  return tensor , tokenizer

In [ ]:
#Drivers code
input_tensor , input_tokenizer = tokenize(input_texts)
target_tensor , target_tokenizer = tokenize(target_texts)

In [ ]:
#Vocabulary sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

In [ ]:
#Sequences lengths
max_input_len = input_tensor.shape[1]
max_target_len = target_tensor.shape[1]

In [ ]:
#Encoder - decoder with LSTM
embedding_dim = 256
units = 1024

In [ ]:
#Encoder
encoder_inputs = Input(shape = (None,))
enc_emb = Embedding(input_vocab_size , embedding_dim , trainable = True)(encoder_inputs)
encoder_lstm = LSTM(units , return_state = True)
encoder_outputs , state_h , state_c = encoder_lstm(enc_emb)
encoder_states = [state_h , state_c]

In [ ]:
#Dcoder
decoder_inputs = Input(shape = (None,))
dec_emb = Embedding(target_vocab_size , embedding_dim )(decoder_inputs)
decoder_lstm = LSTM(units , return_sequences = True , return_state = True)
decoder_outputs , _ , _ = decoder_lstm(dec_emb , initial_state = encoder_states)
decoder_dense = Dense(target_vocab_size , activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#Define the model
model = Model([encoder_inputs , decoder_inputs] , decoder_outputs)

In [ ]:
#Compile the model
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
#Prepare decoder target data by shifting
target_input = target_tensor[: , :-1]
target_output = target_tensor[: , 1:]
target_output = np.expand_dims(target_output , axis = -1)

In [ ]:
#Train the model
model.fit([input_tensor , target_input] , target_output , batch_size = 32 , epochs = 3)

Epoch 1/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.2970 - loss: 5.9672
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step - accuracy: 0.3492 - loss: 4.2918
Epoch 3/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.3644 - loss: 4.0581


In [ ]:
#Model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │  1,006,848 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 256) │    847,616 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 1024),    │  5,246,976 │ embedding[0][0]   │
│                     │ (None, 1024),     │            │                   │
│                     │ (None, 1024)]     │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, None,     │  5,246,976 │ embedding_2[0][0… │
│                     │ 1024), (None,     │            │ lstm[0][1],       │
│                     │ 1024), (None,     │            │ lstm[0][2]        │
│                     │ 1024)]            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  3,393,775 │ lstm_2[0][0]      │
│                     │ 3311)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 47,226,575 (180.16 MB)

 Trainable params: 15,742,191 (60.05 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 31,484,384 (120.10 MB)